Yellow/Grin Technical Challenge Data Team
===========

In [14]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import psycopg2
import seaborn as sns

from geopy import Point
from geopy.distance import geodesic

### Connecting to Grin Challenge Database


In [4]:
def get_from_sql(query):
    conn = psycopg2.connect(user='grin_technical_challenge',
                               host='grin-technical-test.crf3v8rvj0hp.us-east-1.rds.amazonaws.com',
                               database='postgres',
                               port='5432',
                               password='kjbaskc24bkjhb354bbjk')
    df = pd.read_sql(query, conn)
    return df

### Retrieving rides data and joining with open app events with session.


In [5]:
query = '''
select rd.id as ride_id
     , rd.start_time
     , rd.finish_time
     , rd.starting_lng
     , rd.starting_lat
     , rd.dest_lng
     , rd.dest_lat
     , rd.price
     , rd.scooter_id
     , oa.open_time
     , oa.id as open_app_id
     , oa.starting_lng as open_app_lng
     , oa.starting_lat as open_app_lat
from public.rides rd
left join public.open_app oa
on rd.open_app_id = oa.id
'''


df = get_from_sql(query)

Calculate distance between the user when he opened the app and the scooter chosen to ride

In [6]:
def get_geodesic_distance(point1, point2):
    """
    Measure the geodesic distance in meters between 2 points.
    >>> get_geodesic_distance((-23.56768, -46.6789234), (-23.57768, -46.6739234))
    1219.4818091223003

    :param point1: tuple containing latitude and longitude pair (in this specific order)
    :param point2: tuple containing latitude and longitude pair (in this specific order)
    
    :return: distance in meters between the given points
    :rtype: float
    """
    try:
        point_a = Point(*point1)
        point_b = Point(*point2)

        return geodesic(point_a, point_b, ellipsoid='WGS-84').meters
    except:
        return None

In [16]:
df['distance_to_scooter'] = df.apply(lambda row: get_geodesic_distance((row['open_app_lat'], row['open_app_lng']), (row['starting_lat'], row['starting_lng'])), axis=1)
df.head()

,ride_id,start_time,finish_time,starting_lng,starting_lat,dest_lng,dest_lat,price,scooter_id,open_time,open_app_id,open_app_lng,open_app_lat,distance_to_scooter
0,22822,2018-08-27 23:59:58.348288,2018-08-28 00:11:44.348288,-46.687651,-23.577836,-46.660098,-23.572932,9.43,40667,2018-08-27 23:59:11.036229,4208314,-46.687651,-23.577836,0.0
1,23077,2018-09-13 21:44:59.369494,2018-09-13 22:10:45.369494,-46.669065,-23.568421,-46.730697,-23.634002,24.19,29374,2018-09-13 21:44:36.178091,7088907,-46.669065,-23.568421,0.0
2,23347,2018-08-17 04:14:18.221686,2018-08-17 04:36:44.221686,-46.542360,-23.528990,-46.566309,-23.539473,12.77,117637,2018-08-17 04:14:04.173611,2527218,-46.542360,-23.528990,0.0
3,23413,2018-08-22 09:08:16.162486,2018-08-22 09:20:01.162486,-46.533701,-23.650436,-46.544000,-23.640647,6.00,88000,2018-08-22 09:07:30.332329,3324081,-46.533701,-23.650436,0.0
4,24235,2018-09-01 02:29:44.037555,2018-09-01 02:45:12.037555,-46.570279,-23.742442,-46.598598,-23.751569,9.90,6779,2018-09-01 02:29:07.982184,4891295,-46.570279,-23.742442,0.0


Same distance. Can't generate any insights from this.